In [35]:
import carla
import random
import time
import networkx as nx

import numpy as np
import pandas as pd
import utility as util
import carlautil
HOST = '127.0.0.1'
PORT = 2000

In [36]:
client = carla.Client(HOST, PORT)

"""Get the current world."""
carla_world = client.get_world()
carla_map = carla_world.get_map()
carla_map.name

'Town10HD'

Landmarks include traffic lights, stop signs, and speed limit signs.
There is no way I know of to find the speed limit besides obtaining these signs.

In [37]:
## LANDMARKS
# get all landmark names and IDs
# traffic lights have names Signal_3Light_Post01
landmark_names = set()
landmark_types = set()

for lm in carla_map.get_all_landmarks():
    landmark_names.add(lm.name)
    landmark_types.add(lm.type)
    if 'Speed_' in lm.name:
        carlautil.debug_point(client, lm.transform, t=15.0)
    
print("landmark names:", *landmark_names)
print("landmark type IDs:", *landmark_types)

landmark names:  Stencil_STOP Signal_3Light_Post01 Sign_Yield Sign_Stop
landmark type IDs: 206 205 1000001


In [39]:
## TRAFFIC LIGHTS
# getting traffic lights
tlights = filter(lambda a: 'traffic_light' in a.type_id, carla_world.get_actors())


224.50999450683594

In [26]:
## TOPOLOGY and JUNCTION
# select uncontrolled junctions (no traffic lights) by measuring proximity of junctions and traffic lights.
tlights = util.filter_to_list(lambda a: 'traffic_light' in a.type_id,
        carla_world.get_actors())
topology = carla_map.get_topology()
G = nx.Graph()
G.add_edges_from(topology)
junctions = util.get_junctions_from_topology_graph(G)
print(len(tlights), "traffic lights in map")
print(len(junctions), "unique junctions in map")

tlight_distances = np.zeros((len(tlights), len(junctions),))
f = lambda j: util.location_to_ndarray(j.bounding_box.location)
junction_locations = util.map_to_ndarray(f, junctions)
# print(junction_locations)
f = lambda tl: util.transform_to_location_ndarray(
        tl.get_transform())
tlight_locations = util.map_to_ndarray(f, tlights)
# print(tlight_locations)
for idx, junction in enumerate(junctions):
    tlight_distances[:,idx] = np.linalg.norm(tlight_locations - junction_locations[idx], axis=1)

is_uncontrolled_junction = np.logical_not((tlight_distances < 25).any(axis=0))
uncontrolled_junctions = util.compress_to_list(junctions, is_uncontrolled_junction)
print(len(uncontrolled_junctions), "uncontrolled junctions")

38 traffic lights in map
31 unique junctions in map
20 uncontrolled junctions
